In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
from software.preprocessing.video_data.DLC.Reconstruction import dlt_reconstruct
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.metrics import mean_squared_error
import shutil, os
from PIL import Image
from errno import EEXIST, ENOENT
import pdb

In [2]:
splits = ['10', '20', '30', '40', '50', '80', '90']

prediction_csvs = {split: {} for split in splits}

for split in splits:
    prediction_csvs[split]['rm10'] = ['/Users/bassp/OneDrive/Desktop/Brett Stuff/Resnet101_AllCam_GroundTruths/Split_{}/ground_truths_C1_RM10DLC_resnet101_LabLabelOct27shuffle1_500000.csv'.format(split), 'C:/Users/bassp/OneDrive/Desktop/Brett Stuff/Resnet101_AllCam_GroundTruths/Split_{}/ground_truths_cam2_RM10DLC_resnet101_LabLabelOct27shuffle1_500000.csv'.format(split), 'C:/Users/bassp/OneDrive/Desktop/Brett Stuff/Resnet101_AllCam_GroundTruths/Split_{}/ground_truths_cam3_RM10DLC_resnet101_LabLabelOct27shuffle1_500000.csv'.format(split)]

    prediction_csvs[split]['rm11'] = ['/Users/bassp/OneDrive/Desktop/Brett Stuff/Resnet101_AllCam_GroundTruths/Split_{}/ground_truths_cam1_RM11DLC_resnet101_LabLabelOct27shuffle1_500000.csv'.format(split), 'C:/Users/bassp/OneDrive/Desktop/Brett Stuff/Resnet101_AllCam_GroundTruths/Split_{}/ground_truths_cam2_RM11DLC_resnet101_LabLabelOct27shuffle1_500000.csv'.format(split), 'C:/Users/bassp/OneDrive/Desktop/Brett Stuff/Resnet101_AllCam_GroundTruths/Split_{}/ground_truths_cam3_RM11DLC_resnet101_LabLabelOct27shuffle1_500000.csv'.format(split)]

    prediction_csvs[split]['rm13'] = ['/Users/bassp/OneDrive/Desktop/Brett Stuff/Resnet101_AllCam_GroundTruths/Split_{}/ground_truths_cam1_RM13DLC_resnet101_LabLabelOct27shuffle1_500000.csv'.format(split), 'C:/Users/bassp/OneDrive/Desktop/Brett Stuff/Resnet101_AllCam_GroundTruths/Split_{}/ground_truths_cam2_RM13DLC_resnet101_LabLabelOct27shuffle1_500000.csv'.format(split), 'C:/Users/bassp/OneDrive/Desktop/Brett Stuff/Resnet101_AllCam_GroundTruths/Split_{}/ground_truths_cam3_RM13DLC_resnet101_LabLabelOct27shuffle1_500000.csv'.format(split)]

rm10_label_csvs = ['/Users/bassp/OneDrive/Desktop/Brett Stuff/March Labels/CollectedData_Lab_cam1_RM10.csv', '/Users/bassp/OneDrive/Desktop/Brett Stuff/March Labels/CollectedData_Lab_cam2_RM10.csv', '/Users/bassp/OneDrive/Desktop/Brett Stuff/March Labels/CollectedData_Lab_cam3_RM10.csv']
rm11_label_csvs = ['/Users/bassp/OneDrive/Desktop/Brett Stuff/March Labels/CollectedData_Lab_cam1_RM11.csv', '/Users/bassp/OneDrive/Desktop/Brett Stuff/March Labels/CollectedData_Lab_cam2_RM11.csv', '/Users/bassp/OneDrive/Desktop/Brett Stuff/March Labels/CollectedData_Lab_cam3_RM11.csv']
rm13_label_csvs = ['/Users/bassp/OneDrive/Desktop/Brett Stuff/March Labels/CollectedData_Lab_cam1_RM13.csv', '/Users/bassp/OneDrive/Desktop/Brett Stuff/March Labels/CollectedData_Lab_cam2_RM13.csv', '/Users/bassp/OneDrive/Desktop/Brett Stuff/March Labels/CollectedData_Lab_cam3_RM13.csv']

In [3]:
def process_csv(prediction_csvs, label_csvs):
    data_dict = {}
    cam1_csv = np.loadtxt(label_csvs[0], dtype=str, delimiter=",")
    names = cam1_csv[1,1::2].copy()

    cam1_dlc_data = np.loadtxt(prediction_csvs[0], skiprows=3, delimiter=",")[:, 1:]
    cam2_dlc_data = np.loadtxt(prediction_csvs[1], skiprows=3, delimiter=",")[:, 1:]
    cam3_dlc_data = np.loadtxt(prediction_csvs[2], skiprows=3, delimiter=",")[:, 1:]

    cam1_csv_data = np.genfromtxt(label_csvs[0], delimiter=",", skip_header=3, usecols=range(1, len(names)*2 + 1))
    cam2_csv_data = np.genfromtxt(label_csvs[1], delimiter=",", skip_header=3, usecols=range(1, len(names)*2 + 1))
    cam3_csv_data = np.genfromtxt(label_csvs[2], delimiter=",", skip_header=3, usecols=range(1, len(names)*2 + 1))

    discarded = set()
    shortest_len = min(cam1_dlc_data.shape[0], cam2_dlc_data.shape[0], cam3_dlc_data.shape[0])
    for i in range(len(names)):
        labelled_idx = i * 2
        predicted_idx = i * 3
        entry_dict = {}

        cam1_labelled_x = cam1_csv_data[:shortest_len, labelled_idx].copy()
        cam1_predicted_x = cam1_dlc_data[:shortest_len, predicted_idx].copy()
        cam1_labelled_y = cam1_csv_data[:shortest_len, labelled_idx + 1].copy()
        cam1_predicted_y = cam1_dlc_data[:shortest_len, predicted_idx + 1].copy()
        cam1_predicted_prob = cam1_dlc_data[:shortest_len, predicted_idx + 2].copy()
        
        cam2_labelled_x = cam2_csv_data[:shortest_len, labelled_idx].copy()
        cam2_predicted_x = cam2_dlc_data[:shortest_len, predicted_idx].copy()
        cam2_labelled_y = cam2_csv_data[:shortest_len, labelled_idx + 1].copy()
        cam2_predicted_y = cam2_dlc_data[:shortest_len, predicted_idx + 1].copy()
        cam2_predicted_prob = cam2_dlc_data[:shortest_len, predicted_idx + 2].copy()

        cam3_labelled_x = cam3_csv_data[:shortest_len, labelled_idx].copy()
        cam3_predicted_x = cam3_dlc_data[:shortest_len, predicted_idx].copy()
        cam3_labelled_y = cam3_csv_data[:shortest_len, labelled_idx + 1].copy()
        cam3_predicted_y = cam3_dlc_data[:shortest_len, predicted_idx + 1].copy()
        cam3_predicted_prob = cam3_dlc_data[:shortest_len, predicted_idx + 2].copy()


        mask = np.isnan(cam1_labelled_x) | np.isnan(cam1_labelled_y) | np.isnan(cam2_labelled_x) | np.isnan(cam2_labelled_y) | np.isnan(cam3_labelled_x) | np.isnan(cam3_labelled_y)
        if np.sum(~mask) < 10:
            #print("Not enough entries for {}, discarding".format(names[i]))
            discarded.add(names[i])
            continue
        cam1_labelled_x[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), cam1_labelled_x[~mask])
        cam1_labelled_y[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), cam1_labelled_y[~mask])
        cam2_labelled_x[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), cam2_labelled_x[~mask])
        cam2_labelled_y[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), cam2_labelled_y[~mask])
        cam3_labelled_x[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), cam3_labelled_x[~mask])
        cam3_labelled_y[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), cam3_labelled_y[~mask])
        
        entry_dict["Camera 1"] = {}
        entry_dict["Camera 1"]["labelled_x"] = cam1_labelled_x
        entry_dict["Camera 1"]["predicted_x"] = cam1_predicted_x
        entry_dict["Camera 1"]["labelled_y"] = cam1_labelled_y
        entry_dict["Camera 1"]["predicted_y"] = cam1_predicted_y
        entry_dict["Camera 1"]["predicted_prob"] = cam1_predicted_prob
        entry_dict["Camera 1"]["max_x"] = max(np.max(entry_dict["Camera 1"]["labelled_x"]), np.max(entry_dict["Camera 1"]["predicted_x"]))
        entry_dict["Camera 1"]["max_y"] = max(np.max(entry_dict["Camera 1"]["labelled_y"]), np.max(entry_dict["Camera 1"]["predicted_y"]))
        entry_dict["Camera 1"]["min_x"] = min(np.min(entry_dict["Camera 1"]["labelled_x"]), np.min(entry_dict["Camera 1"]["predicted_x"]))
        entry_dict["Camera 1"]["min_y"] = min(np.min(entry_dict["Camera 1"]["labelled_y"]), np.min(entry_dict["Camera 1"]["predicted_y"]))

        entry_dict["Camera 2"] = {}
        entry_dict["Camera 2"]["labelled_x"] = cam2_labelled_x
        entry_dict["Camera 2"]["predicted_x"] = cam2_predicted_x
        entry_dict["Camera 2"]["labelled_y"] = cam2_labelled_y
        entry_dict["Camera 2"]["predicted_y"] = cam2_predicted_y
        entry_dict["Camera 2"]["predicted_prob"] = cam2_predicted_prob
        entry_dict["Camera 2"]["max_x"] = max(np.max(entry_dict["Camera 2"]["labelled_x"]), np.max(entry_dict["Camera 2"]["predicted_x"]))
        entry_dict["Camera 2"]["max_y"] = max(np.max(entry_dict["Camera 2"]["labelled_y"]), np.max(entry_dict["Camera 2"]["predicted_y"]))
        entry_dict["Camera 2"]["min_x"] = min(np.min(entry_dict["Camera 2"]["labelled_x"]), np.min(entry_dict["Camera 2"]["predicted_x"]))
        entry_dict["Camera 2"]["min_y"] = min(np.min(entry_dict["Camera 2"]["labelled_y"]), np.min(entry_dict["Camera 2"]["predicted_y"]))

        entry_dict["Camera 3"] = {}
        entry_dict["Camera 3"]["labelled_x"] = cam3_labelled_x
        entry_dict["Camera 3"]["predicted_x"] = cam3_predicted_x
        entry_dict["Camera 3"]["labelled_y"] = cam3_labelled_y
        entry_dict["Camera 3"]["predicted_y"] = cam3_predicted_y
        entry_dict["Camera 3"]["predicted_prob"] = cam3_predicted_prob
        entry_dict["Camera 3"]["max_x"] = max(np.max(entry_dict["Camera 3"]["labelled_x"]), np.max(entry_dict["Camera 3"]["predicted_x"]))
        entry_dict["Camera 3"]["max_y"] = max(np.max(entry_dict["Camera 3"]["labelled_y"]), np.max(entry_dict["Camera 3"]["predicted_y"]))
        entry_dict["Camera 3"]["min_x"] = min(np.min(entry_dict["Camera 3"]["labelled_x"]), np.min(entry_dict["Camera 3"]["predicted_x"]))
        entry_dict["Camera 3"]["min_y"] = min(np.min(entry_dict["Camera 3"]["labelled_y"]), np.min(entry_dict["Camera 3"]["predicted_y"]))
        data_dict[names[i]] = entry_dict
    filtered_names = [name for name in names if name not in discarded]
    data_dict["filtered_names"] = filtered_names
    thres = 0.05
    for name in filtered_names:
        for cam in ["Camera 1", "Camera 2", "Camera 3"]:
            mask = data_dict[name][cam]["predicted_prob"] < thres
            data_dict[name][cam]["interp_predicted_prob"] = data_dict[name][cam]["predicted_prob"].copy()
            data_dict[name][cam]["interp_predicted_prob"][mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), data_dict[name][cam]["predicted_prob"][~mask])
            data_dict[name][cam]["interp_predicted_x"] = data_dict[name][cam]["predicted_x"].copy()
            data_dict[name][cam]["interp_predicted_x"][mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), data_dict[name][cam]["predicted_x"][~mask])
            data_dict[name][cam]["interp_predicted_y"] = data_dict[name][cam]["predicted_y"].copy()
            data_dict[name][cam]["interp_predicted_y"][mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), data_dict[name][cam]["predicted_y"][~mask])
    return data_dict

In [4]:
def get_reconstructions(data_dict, dlt_coefs_file):
    dlt_coefs = np.loadtxt(dlt_coefs_file, delimiter=",")
    reconstructions = {}
    for name in data_dict["filtered_names"]:
        # read in data from DLC
        frames = data_dict[name]["Camera 1"]["labelled_x"].shape[0]
        cam_data = np.empty([frames, 6], dtype=float)
        weights = np.empty([frames, 6], dtype=float)

        cam_data[:, 0] = data_dict[name]["Camera 1"]["labelled_x"]
        cam_data[:, 1] = data_dict[name]["Camera 1"]["labelled_y"]
        cam_data[:, 2] = data_dict[name]["Camera 2"]["labelled_x"]
        cam_data[:, 3] = data_dict[name]["Camera 2"]["labelled_y"]
        cam_data[:, 4] = data_dict[name]["Camera 3"]["labelled_x"]
        cam_data[:, 5] = data_dict[name]["Camera 3"]["labelled_y"]

        xyz_labelled = dlt_reconstruct(dlt_coefs, cam_data)

        cam_data[:, 0] = data_dict[name]["Camera 1"]["predicted_x"]
        cam_data[:, 1] = data_dict[name]["Camera 1"]["predicted_y"]
        cam_data[:, 2] = data_dict[name]["Camera 2"]["predicted_x"]
        cam_data[:, 3] = data_dict[name]["Camera 2"]["predicted_y"]
        cam_data[:, 4] = data_dict[name]["Camera 3"]["predicted_x"]
        cam_data[:, 5] = data_dict[name]["Camera 3"]["predicted_y"]
        
        weights[:, 0] = data_dict[name]["Camera 1"]["predicted_prob"]
        weights[:, 1] = data_dict[name]["Camera 1"]["predicted_prob"]
        weights[:, 2] = data_dict[name]["Camera 2"]["predicted_prob"]
        weights[:, 3] = data_dict[name]["Camera 2"]["predicted_prob"]
        weights[:, 4] = data_dict[name]["Camera 3"]["predicted_prob"]
        weights[:, 5] = data_dict[name]["Camera 3"]["predicted_prob"]

        xyz_predicted_weighted = dlt_reconstruct(dlt_coefs, cam_data, weights)
        xyz_predicted_unweighted = dlt_reconstruct(dlt_coefs, cam_data)

        cam_data[:, 0] = data_dict[name]["Camera 1"]["interp_predicted_x"]
        cam_data[:, 1] = data_dict[name]["Camera 1"]["interp_predicted_y"]
        cam_data[:, 2] = data_dict[name]["Camera 2"]["interp_predicted_x"]
        cam_data[:, 3] = data_dict[name]["Camera 2"]["interp_predicted_y"]
        cam_data[:, 4] = data_dict[name]["Camera 3"]["interp_predicted_x"]
        cam_data[:, 5] = data_dict[name]["Camera 3"]["interp_predicted_y"]
        
        weights[:, 0] = data_dict[name]["Camera 1"]["interp_predicted_prob"]
        weights[:, 1] = data_dict[name]["Camera 1"]["interp_predicted_prob"]
        weights[:, 2] = data_dict[name]["Camera 2"]["interp_predicted_prob"]
        weights[:, 3] = data_dict[name]["Camera 2"]["interp_predicted_prob"]
        weights[:, 4] = data_dict[name]["Camera 3"]["interp_predicted_prob"]
        weights[:, 5] = data_dict[name]["Camera 3"]["interp_predicted_prob"]

        xyz_interp_predicted_weighted = dlt_reconstruct(dlt_coefs, cam_data, weights)
        xyz_interp_predicted_unweighted = dlt_reconstruct(dlt_coefs, cam_data)

        reconstructions[name] = {}
        reconstructions[name]["xyz_labelled"] = xyz_labelled
        reconstructions[name]["xyz_predicted_weighted"] = xyz_predicted_weighted
        reconstructions[name]["xyz_predicted_unweighted"] = xyz_predicted_unweighted
        reconstructions[name]["xyz_interp_predicted_weighted"] = xyz_interp_predicted_weighted
        reconstructions[name]["xyz_interp_predicted_unweighted"] = xyz_interp_predicted_unweighted
        reconstructions[name]["xyz_predicted_prob"] = np.mean(np.array([data_dict[name]["Camera 1"]["predicted_prob"], data_dict[name]["Camera 2"]["predicted_prob"], data_dict[name]["Camera 3"]["predicted_prob"]]), axis=0)
        reconstructions[name]["xyz_interp_predicted_prob"] = np.mean(np.array([data_dict[name]["Camera 1"]["interp_predicted_prob"], data_dict[name]["Camera 2"]["interp_predicted_prob"], data_dict[name]["Camera 3"]["interp_predicted_prob"]]), axis=0)
    return reconstructions

In [6]:
dlt_coefs_file = '/Users/bassp/OneDrive/Desktop/Brett Stuff/DeepLabCutCSVs/test_dlt.csv'
reconstructions = {}
for split in splits:
    rm10_data_dict = process_csv(prediction_csvs[split]['rm10'], rm10_label_csvs)
    rm11_data_dict = process_csv(prediction_csvs[split]['rm11'], rm11_label_csvs)
    rm13_data_dict = process_csv(prediction_csvs[split]['rm13'], rm13_label_csvs)
   # pdb.set_trace()
    rm10_reconstructions = get_reconstructions(rm10_data_dict, dlt_coefs_file)
    rm11_reconstructions = get_reconstructions(rm11_data_dict, dlt_coefs_file)
    rm13_reconstructions = get_reconstructions(rm13_data_dict, dlt_coefs_file)
    reconstructions[split] = {'rm10':rm10_reconstructions, 'rm11':rm11_reconstructions, 'rm13':rm13_reconstructions}

In [7]:
def mkdir_p(mypath):
    '''Creates a directory. equivalent to using mkdir -p on the command line'''
    try:
        os.makedirs(mypath)
    except OSError as exc: # Python >2.5
        if exc.errno == EEXIST and os.path.isdir(mypath):
            pass
        else: raise

def rm_dir(mypath):
    '''Deletes a directory. equivalent to using rm -rf on the command line'''
    try:
        shutil.rmtree(mypath)
    except OSError as exc: # Python >2.5
        if exc.errno == ENOENT:
            pass
        else: raise

In [9]:
rm_dir('3d')
def visualize_3D_data(reconstructions, split, data_type = "xyz_labelled"):
    for rat in reconstructions[split].keys():
        try:
            xmax, ymax, zmax = np.max([np.max(reconstructions[split][rat][part][data_type], axis=0) for part in reconstructions[split][rat].keys()], axis=0)
            xmin, ymin, zmin = np.min([np.min(reconstructions[split][rat][part][data_type], axis=0) for part in reconstructions[split][rat].keys()], axis=0)
        except:
            pdb.set_trace()
        for i in range(reconstructions[split][rat]["Handle"][data_type].shape[0]):
            if 'pred' in data_type:
                output_dir = '3d/{}/predicted'.format(rat)
            else:
                output_dir = '3d/{}'.format(rat)
            mkdir_p(output_dir)
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            ax.set_xlim(xmax, xmin)
            ax.set_ylim(ymax, ymin)
            ax.set_zlim(zmax, zmin)
            for name in reconstructions[split][rat].keys():
                ax.scatter(*reconstructions[split][rat][name][data_type][i], label=name)
            fig.legend()
            filename='{}/3d_vis_rat_{}_frame{}.png'.format(output_dir, rat, i)
            plt.savefig(filename, dpi=100)
            plt.show()
            #plt.close()


In [ ]:
# "xyz_interp_predicted_prob"
def ground_truth_3D(reconstructionss,split):
    visualize_3D_data(reconstructionss,split)
    for rat in reconstructionss[split].keys():
        output_dir = '3d/{}'.format(rat)
        png_count = len([name for name in os.listdir(output_dir) if os.path.isfile(os.path.join(output_dir, name))])
        files = []
        for i in range(png_count):
            filename='{}/3d_vis_rat_{}_frame{}.png'.format(output_dir, rat, i)
            files.append(filename)

        frames = []
        for i in files:
            new_frame = Image.open(i)
            frames.append(new_frame)

        # Save into a GIF file that loops forever   
        frames[0].save('{}/3d_vis_rat_{}.gif'.format('3d', rat), format='GIF',
                    append_images=frames[1:],
                    save_all=True,
                    duration=40, loop=0)
ground_truth_3D(reconstructions,split)

In [ ]:
interp = "xyz_interp_predicted_prob"
sll = "xyz_predicted_prob"

def predicted_3D_gifs(reconstructions,split):
    visualize_3D_data(reconstructions,split, data_type = "xyz_predicted_prob")
    for rat in reconstructions[split].keys():
        output_dir = '3d/{}/predicted'.format(rat)
        png_count = len([name for name in os.listdir(output_dir) if os.path.isfile(os.path.join(output_dir, name))])
        files = []
        for i in range(png_count):
            filename='{}/3d_vis_rat_{}_frame{}.png'.format(output_dir, rat, i)
            files.append(filename)

        frames = []
        for i in files:
            new_frame = Image.open(i)
            frames.append(new_frame)

        # Save into a GIF file that loops forever   
        frames[0].save('{}/3d_vis_predicted_rat_{}.gif'.format('3d', rat), format='GIF',
                    append_images=frames[1:],
                    save_all=True,
                    duration=40, loop=0)

        
predicted_3D_gifs(reconstructions,split)

In [ ]:
errors = {}
for split in splits:
    split_errors = {}
    total_errors = {}
    part_probabilities = {}
    for rat, reconstruction in reconstructions[split].items():
        rat_errors = []
        for name in reconstruction.keys():
            body_part_error = 1000 * mean_squared_error(reconstruction[name]["xyz_labelled"].T, reconstruction[name]["xyz_predicted_weighted"].T, squared=False, multioutput='raw_values')
            rat_errors.append(body_part_error)
            if total_errors.get(name) is None:
                total_errors[name] = []
                part_probabilities[name] = []
            total_errors[name].extend(body_part_error)
            part_probabilities[name].extend(reconstruction[name]["xyz_predicted_prob"])
        split_errors[rat] = np.array(rat_errors)
    errors[split] = {"by_rat" : split_errors, "by_part" : total_errors, "part_prob" : part_probabilities}

In [ ]:
fig = plt.figure(figsize=(50,7))
ax = fig.add_subplot(111)
start_pos = 1
for i in range(len(splits)):
    num_parts, _ = errors[splits[i]]["by_rat"]['rm10'].shape
    ax.boxplot(errors[splits[i]]["by_rat"]['rm10'].T, positions=range(start_pos, start_pos + num_parts), labels=reconstructions[splits[i]]['rm10'].keys(), showfliers=False)
    start_pos = start_pos + num_parts + 2
ax.set_title("RMSE Distibution across Body Parts (Rat 10)")
fig.autofmt_xdate(rotation=45)

In [ ]:
fig = plt.figure(figsize=(50,7))
ax = fig.add_subplot(111)
start_pos = 1
for i in range(len(splits)):
    labels, data = errors[splits[i]]["by_part"].keys(), errors[splits[i]]["by_part"].values()
    ax.boxplot(data, labels=labels, positions=range(start_pos, start_pos + len(labels)), showfliers=False)
    start_pos = start_pos + len(labels) + 2
ax.set_title("RMSE Distibution across Body Parts")
fig.autofmt_xdate(rotation=45)

In [ ]:
mean_data = np.array([[np.mean(values) for values in errors[split]["by_part"].values()] for split in splits]).T

fig = plt.figure()
ax = fig.add_subplot(111)
for i in range(len(splits)):
    ax.plot(mean_data[:,i], label=splits[i])
ax.legend()
ax.set_title("Mean error per body part")
ax.set_xlabel("Body part")
ax.set_ylabel("Error (mm)")
ax.set_xticks(range(len(errors[split]["by_part"].keys())))
ax.set_xticklabels(errors[split]["by_part"].keys())
fig.autofmt_xdate(rotation=45)

In [ ]:
for part in errors[split]["by_part"].keys():
    fig = plt.figure(figsize=(50,7))
    fig.suptitle("RMSE Split {} vs All Others ({})".format(splits[-1], part))
    for i in range(len(splits) - 1):
        ax = fig.add_subplot(1, len(splits) - 1, i + 1)
        x, y = np.array(errors[splits[-1]]["by_part"][part]), np.array(errors[splits[i]]["by_part"][part])
        vmax = max(np.max(x), np.max(y))
        pad = 0.1 * vmax
        ax.set_ylim(0, vmax + pad)
        ax.set_xlim(0, vmax + pad)
        ax.set_aspect('equal')
        ax.scatter(x, y, color='r')
        ax.plot([ax.get_ylim()[0], ax.get_ylim()[1]],[ax.get_ylim()[0], ax.get_ylim()[1]],'k--', lw=1) # identity line
        ax.set_title("Split {} vs Split {}".format(splits[-1], splits[i]))
        ax.set_ylabel("Split {} RMSE (mm)".format(splits[i]))
        ax.set_xlabel("Split {} RMSE (mm)".format(splits[-1]))

In [ ]:
for part in errors[split]["by_part"].keys():
    fig = plt.figure(figsize=(50,7))
    fig.suptitle("RMSE Split {} vs All Others ({})".format(splits[-1], part))
    for i in range(len(splits)):
        ax = fig.add_subplot(1, len(splits), i + 1)
        ax.set_ylim(0, 100)
        x, y = np.array(errors[splits[i]]["by_part"][part]), 100 * np.array(errors[splits[i]]["part_prob"][part])
        ax.scatter(x, y, color='r')
        ax.set_title("RMSE vs Mean Camera Probability (Split {})".format(splits[i]))
        ax.set_ylabel("Probability")
        ax.set_xlabel("Split {} RMSE (mm)".format(splits[-1]))